In [ ]:
# -------------------------------
# FIX NUMPY/SCIPY ISSUES
# -------------------------------
!pip install -q numpy==1.26.4 scipy==1.11.4
!pip install -q gensim pyLDAvis spacy nltk matplotlib

# Download spaCy model
!python -m spacy download en_core_web_sm

# Restart runtime automatically after fixing dependencies
import os, sys
os.kill(os.getpid(), 9)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# -------------------------------
# Imports & NLTK downloads
# -------------------------------
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')    # ✅ Fix for new NLTK versions
nltk.download('stopwords')
nltk.download('omw-1.4')

import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])  # only tagger+lemmatizer

import gensim
from gensim import corpora
from gensim.models import Phrases, LdaModel
from gensim.models.phrases import Phraser

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# -------------------------------
# Sample corpus (social media posts)
# -------------------------------
corpus = [
    "Just tried the new cafe downtown — the coffee was amazing and the ambiance perfect.",
    "Feeling blessed to have completed my first 5K run today! #fitness #goals",
    "Major update: the latest smartphone from TechCorp features a 108MP camera and longer battery life.",
    "Elections next month — make sure to register and vote. Democracy matters.",
    "Watching the championship game tonight, go Tigers! What a comeback in the second half.",
    "I'm finally starting my online data science course. Excited and nervous!",
    "New study shows climate change is accelerating polar ice melt.",
    "Anyone recommends a good budget laptop for programming and gaming?",
    "Baking banana bread during lockdown. Smells so good!",
    "Stock markets rally after positive earnings reports from major banks."
]

# -------------------------------
# 1) Preprocessing function
# -------------------------------
stop_words = set(stopwords.words('english'))

def preprocess_texts(texts, nlp_pipeline):
    processed_texts = []
    for doc in texts:
        tokens = word_tokenize(doc.lower())
        tokens = [t for t in tokens if t.isalpha() and t not in stop_words and len(t) > 2]
        spacy_doc = nlp_pipeline(" ".join(tokens))
        lemmas = [token.lemma_ for token in spacy_doc if token.is_alpha and token.lemma_ not in stop_words and len(token.lemma_) > 2]
        processed_texts.append(lemmas)
    return processed_texts

processed = preprocess_texts(corpus, nlp)
print("Sample preprocessed docs:", processed[:3])

# -------------------------------
# 2) Create bigrams
# -------------------------------
bigram = Phrases(processed, min_count=2, threshold=5)
bigram_mod = Phraser(bigram)
data_bigrams = [bigram_mod[doc] for doc in processed]

# -------------------------------
# 3) Dictionary + Corpus
# -------------------------------
id2word = corpora.Dictionary(data_bigrams)
id2word.filter_extremes(no_below=1, no_above=0.5)
gensim_corpus = [id2word.doc2bow(text) for text in data_bigrams]

# -------------------------------
# 4) LDA model
# -------------------------------
num_topics = 3
lda_model = LdaModel(corpus=gensim_corpus,
                     id2word=id2word,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     passes=10,
                     alpha='auto')

print("\nLDA topics:")
for idx, topic in lda_model.print_topics(num_topics=num_topics, num_words=6):
    print(f"Topic {idx}: {topic}")

# -------------------------------
# 5) Interactive visualization
# -------------------------------
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, gensim_corpus, id2word)
vis


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Sample preprocessed docs: [['try', 'new', 'cafe', 'downtown', 'coffee', 'amazing', 'ambiance', 'perfect'], ['feel', 'bless', 'complete', 'first', 'run', 'today', 'fitness', 'goal'], ['major', 'update', 'late', 'smartphone', 'techcorp', 'feature', 'camera', 'long', 'battery', 'life']]

LDA topics:
Topic 0: 0.013*"election" + 0.013*"vote" + 0.013*"register" + 0.013*"next" + 0.013*"matter" + 0.013*"make"
Topic 1: 0.032*"good" + 0.019*"nervous" + 0.019*"bake" + 0.019*"gaming" + 0.019*"datum" + 0.019*"online"
Topic 2: 0.038*"major" + 0.022*"new" + 0.022*"climate" + 0.022*"late" + 0.022*"long" + 0.022*"smartphone"


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1     -0.094449  0.0       1        1  56.484014
2      0.095874  0.0       2        1  43.035374
0     -0.001425  0.0       3        1   0.480612, topic_info=        Term      Freq     Total Category  logprob  loglift
22     major  1.000000  1.000000  Default  30.0000  30.0000
62      good  1.000000  1.000000  Default  29.0000  29.0000
27  election  1.000000  1.000000  Default  28.0000  28.0000
34      vote  1.000000  1.000000  Default  27.0000  27.0000
32  register  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
70     smell  0.004987  1.043186   Topic3  -4.3575  -0.0053
68     bread  0.004987  1.043193   Topic3  -4.3575  -0.0053
36  comeback  0.004987  1.043093   Topic3  -4.3575  -0.0052
42     watch  0.004987  1.043087   Topic3  -4.3575  -0.0052
45    excite  0.004987  1.043193   Topic3  -4.3575  -0.0053

[148 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
51        2  1.021890  accelerate
59        1  0.958590      anyone
66        1  0.958566        bake
67        1  0.958656      banana
71        2  1.022380        bank
...     ...       ...         ...
15        2  1.021900       today
41        1  0.958693     tonight
25        2  1.021874      update
34        1  0.959739        vote
42        1  0.958693       watch

[72 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
